In [114]:
#-------------------------
# Author: Benjamin Piscoya|24038341
#-------------------------
from sklearn.decomposition import TruncatedSVD
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor
from plotly.subplots import make_subplots
import scipy.sparse as sp, inspect, os, json, joblib, gc, re, io, zipfile, numpy as np, pandas as pd, lightgbm as lgb, xgboost as xgb
import matplotlib.pyplot as plt, nltk, plotly.express as px, plotly.graph_objects as go
from scipy import sparse
from joblib import Parallel, delayed, parallel_backend
from pathlib import Path
from sklearn.manifold import TSNE
from sklearn.model_selection import GroupShuffleSplit, GroupKFold,train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, mean_absolute_error, silhouette_score, calinski_harabasz_score, root_mean_squared_error
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor

from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
#hyperparameter tuning
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

from sklearn.cluster import KMeans 
from collections import Counter
from wordcloud import WordCloud
#---------------------------------------------
from contextlib import redirect_stdout
import warnings
warnings.filterwarnings("ignore")

In [74]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [75]:
# Set display options for pandas
pd.set_option('display.max_columns', None)  # Show all columns
# Global variables
RS_STATE = 123
PATH_FILE = '../data/'

In [76]:
def load_csv_from_zip(
    zip_path: str,
    chunksize: int = 100_000,
    max_chunks: int = None,
    encoding: str = "utf-8",
    fallback_encoding: str = "latin-1",
    errors: str = "replace"
) -> pd.DataFrame:
    """
    Loads a CSV file from a zip archive using chunk-based reading,
    with graceful handling of UnicodeDecodeErrors.

    Parameters
    ----------
    zip_path : str
        Path to the zip file containing exactly one CSV.
    chunksize : int
        Number of rows per chunk.
    max_chunks : int or None
        Maximum number of chunks to read (None = all).
    encoding : str
        Primary text encoding to use.
    fallback_encoding : str
        Fallback encoding if primary fails.
    errors : str
        Error handler for undecodable bytes ('strict', 'replace', 'ignore').

    Returns
    -------
    pd.DataFrame
        Concatenated DataFrame of all loaded chunks.
    """
    # Open the zip and get the first (and only) CSV filename
    with zipfile.ZipFile(zip_path, 'r') as zf:
        csv_name = zf.namelist()[0]
        print(f"Loading '{csv_name}' from '{zip_path}' (chunksize={chunksize})")

        def _open_text(encoding_to_use):
            raw = zf.open(csv_name, 'r')
            return io.TextIOWrapper(raw, encoding=encoding_to_use, errors=errors)

        # Try primary encoding, then fallback if it fails
        try:
            reader = pd.read_csv(_open_text(encoding), chunksize=chunksize)
        except UnicodeDecodeError:
            print(f"Failed to decode with {encoding!r}, retrying with {fallback_encoding!r}")
            reader = pd.read_csv(_open_text(fallback_encoding), chunksize=chunksize)

        # Read chunks
        chunks = []
        for i, chunk in enumerate(reader, start=1):
            chunks.append(chunk)
            print(f"  • Loaded chunk {i}")
            if max_chunks and i >= max_chunks:
                print(f"  • Reached max_chunks={max_chunks}, stopping early")
                break

    # Concatenate and return
    df = pd.concat(chunks, ignore_index=True)
    #print(f"Loaded {len(df)} rows in total")
    return df

In [77]:
attributes_df = load_csv_from_zip(PATH_FILE+"attributes.csv.zip", chunksize=10000)
product_descriptions_df = load_csv_from_zip(PATH_FILE+"product_descriptions.csv.zip", chunksize=10000)
query_df = load_csv_from_zip(PATH_FILE+"train.csv.zip", chunksize=10000)

Loading 'attributes.csv' from '../data/attributes.csv.zip' (chunksize=10000)
  • Loaded chunk 1
  • Loaded chunk 2
  • Loaded chunk 3
  • Loaded chunk 4
  • Loaded chunk 5
  • Loaded chunk 6
  • Loaded chunk 7
  • Loaded chunk 8
  • Loaded chunk 9
  • Loaded chunk 10
  • Loaded chunk 11
  • Loaded chunk 12
  • Loaded chunk 13
  • Loaded chunk 14
  • Loaded chunk 15
  • Loaded chunk 16
  • Loaded chunk 17
  • Loaded chunk 18
  • Loaded chunk 19
  • Loaded chunk 20
  • Loaded chunk 21
  • Loaded chunk 22
  • Loaded chunk 23
  • Loaded chunk 24
  • Loaded chunk 25
  • Loaded chunk 26
  • Loaded chunk 27
  • Loaded chunk 28
  • Loaded chunk 29
  • Loaded chunk 30
  • Loaded chunk 31
  • Loaded chunk 32
  • Loaded chunk 33
  • Loaded chunk 34
  • Loaded chunk 35
  • Loaded chunk 36
  • Loaded chunk 37
  • Loaded chunk 38
  • Loaded chunk 39
  • Loaded chunk 40
  • Loaded chunk 41
  • Loaded chunk 42
  • Loaded chunk 43
  • Loaded chunk 44
  • Loaded chunk 45
  • Loaded chunk 46
  • Loaded c

In [78]:
display(attributes_df.head(4), product_descriptions_df.head(4), query_df.head(4))

,product_uid,name,value
0,100001.0,Bullet01,Versatile connector for various 90° connection...
1,100001.0,Bullet02,Stronger than angled nailing or screw fastenin...
2,100001.0,Bullet03,Help ensure joints are consistently straight a...
3,100001.0,Bullet04,Dimensions: 3 in. x 3 in. x 1-1/2 in.


,product_uid,product_description
0,100001,"Not only do angles make joints stronger, they ..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...
2,100003,Classic architecture meets contemporary design...
3,100004,The Grape Solar 265-Watt Polycrystalline PV So...


,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33


In [79]:
display(attributes_df.info(), product_descriptions_df.info(), query_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2044803 entries, 0 to 2044802
Data columns (total 3 columns):
 #   Column       Dtype  
---  ------       -----  
 0   product_uid  float64
 1   name         object 
 2   value        object 
dtypes: float64(1), object(2)
memory usage: 46.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124428 entries, 0 to 124427
Data columns (total 2 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   product_uid          124428 non-null  int64 
 1   product_description  124428 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74067 entries, 0 to 74066
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             74067 non-null  int64  
 1   product_uid    74067 non-null  int64  
 2   product_title  74067 non-null  object 
 3   search_term 

None

None

None

In [80]:
# nan value check
display(attributes_df.isna().sum(), product_descriptions_df.isna().sum(), query_df.isna().sum())
print(f'Percentage of NaN values in attributes_df: {attributes_df.isna().sum().sum()/attributes_df.size*100:.2f}%')

product_uid     155
name            155
value          6422
dtype: int64

product_uid            0
product_description    0
dtype: int64

id               0
product_uid      0
product_title    0
search_term      0
relevance        0
dtype: int64

Percentage of NaN values in attributes_df: 0.11%


In [81]:
# removing nan values in attributes_df
# 1) 1.1% nan values is trivial, so we can drop them
# 2) the bias generated for a potential imputation outweights the benefit of keeping them due to its low percentage
attributes_df = attributes_df.dropna()

In [82]:
# remove iid column in query_df
query_df = query_df.drop(columns=['id'])

In [83]:
query_df.shape

(74067, 4)

#### EDA Top Most Frequent Search Terms

In [84]:
#Top 10 most frequent search terms
top_terms = query_df['search_term'].value_counts().head(10).reset_index()
top_terms.columns = ['search_term', 'count']


fig = px.bar(
    top_terms.sort_values('count', ascending=False),
    x='count',
    y='search_term',
    orientation='h',
    title='Top 10 Most Frequent Search Terms',
    labels={'count': 'Frequency', 'search_term': 'Search Term'},
    template='plotly_white'
)

fig.update_layout(yaxis=dict(autorange='reversed'))  # Highest freq at top
fig.show()

### Sample DF Pipeline

In [85]:
# --------------------------------------------------------------------------------------------------------
# Due to the large size of the dataset, we will sample 1% to prove that the overall pipeline works properly
# Should you want to run the entire dataset, please comment the first line and uncomment the second one
# ---------------------------------------------------------------------------------------------------------
query_df_sample = query_df.sample(frac=0.01, random_state=RS_STATE)
# query_df_sample = query_df.copy()
query_df_sample.shape

(741, 4)

In [86]:
data_sample = query_df_sample.merge(
    product_descriptions_df,
    on="product_uid",
    how="left",
)
data_sample = data_sample[['product_uid', 'product_title', 'product_description', 'search_term', 'relevance']]

In [87]:
display(data_sample.head(2), data_sample.iloc[0, -1])

,product_uid,product_title,product_description,search_term,relevance
0,110182,Veranda 4 in. x 4 in. x 39 in. Williamsburg De...,The Veranda 4 in. x 4 in. x 39 in. Williamsbur...,stair railing posts anchor,2.33
1,110076,Whirlpool Top Control Dishwasher in Monochroma...,Get clean dishes every time with this Whirlpoo...,stainless steel electric range and microwave,2.67


np.float64(2.33)

#### Preprocessing

In [88]:
# basic text cleaning

def clean_text(t:str)->str:
    t = str(t).lower()
    t = re.sub(r"&\w+;|&#\d+;|<[^>]+>", " ", t)       # strip HTML/entities
    t = re.sub(r"[^\w\s]", " ", t)                   # punctuation → space
    t = re.sub(r"\b\d+(?:\.\d+)?\b", " ", t)       # remove standalone numbers
    return re.sub(r"\s+", " ", t).strip()

# numeric extractor
unit_pat = r"(?P<num>\d+(?:\.\d+)?)\s*(?:in\b|inch(?:es)?\b|cm\b|mm\b|ft\b)"
dim_pat  = r"(?P<num>\d+(?:\.\d+)?)\s*[x×]\s*(?:\d+(?:\.\d+)?)"
noise_re = re.compile(r"(?:#|sku|model|kit|item)\s*\d+", re.I)
CLIP_LOW, CLIP_HIGH = 0, 1_000

def extract_nums(s:str):
    s2 = noise_re.sub(" ", str(s))
    nums = [float(m.group("num")) for m in re.finditer(unit_pat, s2, flags=re.I)]
    nums += [float(m.group("num")) for m in re.finditer(dim_pat, s2, flags=re.I)]
    return nums if nums else [np.nan]

def num_stats(nums):
    a = np.array(nums, dtype=float)
    return a.min(), a.max(), a.mean(), a.max()-a.min()

# NLP tokenisers
snow = SnowballStemmer("english")
lemma = WordNetLemmatizer(); _cache={}

def wn(tag):
    return {'J':wordnet.ADJ,'V':wordnet.VERB,'N':wordnet.NOUN,'R':wordnet.ADV}.get(tag[:1], wordnet.NOUN)

def stem_tok(txt:str):
    return [snow.stem(w) for w in word_tokenize(txt) if len(w)>2]

def lemma_tok(txt:str):
    outs=[]
    for w,p in pos_tag(word_tokenize(txt)):
        if len(w)<3: continue
        key=(w,p)
        if key not in _cache:
            _cache[key]=lemma.lemmatize(w, wn(p))
        outs.append(_cache[key])
    return outs

In [89]:
# Merge tables
q   = query_df_sample
dsc = product_descriptions_df
attr_raw = attributes_df
q   = q[["product_uid","product_title","search_term","relevance"]]
full = q.merge(dsc, on="product_uid", how="left")

# from attr, identify top‑40 attribute names
name_counts = attr_raw["name"].value_counts()
TOP_NAMES = list((name_counts.head(40).index))
print(f"Selecting 40 out of {name_counts.shape[0]} different features as columns in the final table. The rest will be merged into a single text column.")

# split top vs. other rows
attr_top  = attr_raw[attr_raw["name"].isin(TOP_NAMES)].copy()
attr_rest = attr_raw[~attr_raw["name"].isin(TOP_NAMES)].copy()

# pivot top‑40 attributes
attr_top_grp = (
    attr_top.groupby(["product_uid","name"])["value"]
             .apply(lambda vs: " | ".join(sorted(set(vs.astype(str)))))
             .reset_index())
attr_wide = attr_top_grp.pivot(index="product_uid", columns="name", values="value").reset_index()

# build attrs_text from the rest 
attr_rest["kv"] = attr_rest["name"]+": "+attr_rest["value"].astype(str)
attrs_text_df = (
    attr_rest.groupby("product_uid")["kv"]
              .apply(lambda vs:" ".join(vs)).reset_index()
              .rename(columns={"kv":"attrs_text"}))

# merge attributes into main table
full = full.merge(attr_wide, on="product_uid", how="left")
full = full.merge(attrs_text_df, on="product_uid", how="left")
full["attrs_text"] = full["attrs_text"].fillna("")

Selecting 40 out of 5401 different features as columns in the final table. The rest will be merged into a single text column.


In [90]:
# initial transformers
# keep sparse matrix in float32 to save memory
scaler = StandardScaler()
short_vec = TfidfVectorizer(max_features=30_000, stop_words="english", tokenizer=stem_tok,  lowercase=False, dtype=np.float32) 
long_vec  = TfidfVectorizer(max_features=30_000, stop_words="english", tokenizer=lemma_tok, lowercase=False, dtype=np.float32) 
# character 3–5-gram TF-IDF vectoriser
char_vec = TfidfVectorizer(
    analyzer="char",
    ngram_range=(3, 5),
    min_df=2,
    sublinear_tf=True,
    lowercase=True,
    dtype=np.float32 
)

# helpers
def add_numeric_stats(df, src_prefix, txt_series):
    # get min, max, mean, span for every row
    stats = txt_series.apply(lambda x: num_stats(extract_nums(x)))
    cols  = [f"{src_prefix}_{c}" for c in ("min", "max", "mean", "span")]
    stats_df = pd.DataFrame(stats.tolist(),      # 4 numeric cols
                            columns=cols,
                            index=df.index)

    # binary flag → at least one number present
    stats_df[f"{src_prefix}_success"] = stats_df[cols[0]].notna().astype(int)

    # add all 5 columns to df in ONE concat (no fragmentation)
    df = pd.concat([df, stats_df], axis=1)

    return list(stats_df.columns), df          # ← returns new-col list **and** updated df


In [91]:
# ---------- featurisation -------------------------------------
# ------------------------------------------------------------ #
#  SPARSE‑OUTPUT FEATURISER  +  COLUMN‑NAME OPTION             #
# ------------------------------------------------------------ #
def featurise(
    df,
    *,                     # keyword‑only
    fit=False,
    vec_store=None,
    k_bm25=1.2,
    b_bm25=0.75,
    return_columns=False  
):
    """
    Parameters
    ----------
    df : pd.DataFrame
    fit : bool                 fit or just transform vectorisers / scaler
    vec_store : dict | None    external store for fitted objects
    return_columns : bool      if True → return (X, feature_names)

    Returns
    -------
    scipy.sparse.csr_matrix  or  (csr_matrix, list[str])
    """
    # -------------------------------------------------------- #
    # 0) CLEAN
    df = df.copy()
    df["title_clean"] = df["product_title"].fillna("").map(clean_text)
    df["desc_clean"]  = df["product_description"].fillna("").map(clean_text)
    df["attrs_text"]  = df["attrs_text"].fillna("").map(clean_text)

    # -------------------------------------------------------- #
    # 1) NUMERIC STATS
    t_cols, df = add_numeric_stats(df, "t", df["product_title"])
    d_cols, df = add_numeric_stats(df, "d", df["product_description"])

    a_cols = []
    for col in TOP_NAMES:
        if col in df.columns:
            new_cols, df = add_numeric_stats(df, f"a_{col}", df[col].fillna(""))
            a_cols += new_cols
    num_cols = t_cols + d_cols + a_cols   

    # -------------------------------------------------------- #
    # 2) FIT OBJECTS (once)
    store = {} if vec_store is None else vec_store
    if fit or ("idf_s" not in store):
        scaler.fit(df[num_cols])

        short_vec.fit(pd.concat([df["search_term"], df["title_clean"]]))
        long_vec .fit((df["desc_clean"] + " " + df["attrs_text"]).str.strip())
        char_vec .fit(pd.concat([df["search_term"], df["title_clean"]]))

        store.update({
            "idf_s": short_vec.idf_[None, :],
            "idf_l": long_vec.idf_[None, :]
        })
    idf_s, idf_l = store["idf_s"], store["idf_l"]

    # -------------------------------------------------------- #
    # 3) NUMERIC  → CSR
    num_scaled = scaler.transform(df[num_cols]).astype(np.float32)
    num_csr    = sparse.csr_matrix(num_scaled)

    # -------------------------------------------------------- #
    # 4) TEXT VECTORS (CSR)
    q_short = short_vec.transform(df["search_term"])
    tit     = short_vec.transform(df["title_clean"])

    blob    = (df["desc_clean"] + " " + df["attrs_text"]).str.strip()
    lng     = long_vec.transform(blob)
    q_long  = long_vec.transform(df["search_term"])

    q_char  = char_vec.transform(df["search_term"])
    tit_char= char_vec.transform(df["title_clean"])

    # -------------------------------------------------------- #
    # 5) COSINE + BM25  → 1‑col CSR
    colnames_dense = ["cos_t", "cos_l", "bm25_t", "bm25_l", "cos_char"]

    def csr_col(x):
        return sparse.csr_matrix(x.reshape(-1, 1), dtype=np.float32)

    cos_t    = csr_col(q_short.multiply(tit     ).sum(axis=1).A.ravel())
    cos_l    = csr_col(q_long .multiply(lng     ).sum(axis=1).A.ravel())
    cos_char = csr_col(q_char .multiply(tit_char).sum(axis=1).A.ravel())

    def bm25(qm, dm, idf):
        dl   = np.asarray(dm.sum(axis=1)).ravel()
        avdl = dl.mean()
        denom = dm + k_bm25 * (1 - b_bm25 + b_bm25 * dl / avdl)[:, None]
        return (dm.multiply(k_bm25 + 1) / denom).multiply(qm).dot(idf.T)

    bm25_t  = csr_col(bm25(q_short, tit,  idf_s))
    bm25_l  = csr_col(bm25(q_long , lng, idf_l))


    # -------------------------------------------------------- #
    # 6) STACK
    X_sparse = sparse.hstack(
        [cos_t, cos_l, bm25_t, bm25_l, cos_char, num_csr],
        format="csr"
    )

    if return_columns:
        feature_names = colnames_dense + num_cols
        return X_sparse, feature_names
    return X_sparse


#### Train Test Split

In [92]:
# split "full" into dev/test sets
split_outer = GroupShuffleSplit(test_size=0.20, random_state=RS_STATE)
dev_idx, test_idx = next(split_outer.split(full, groups=full["search_term"]))

dev_df   = full.iloc[dev_idx].reset_index(drop=True)
test_df  = full.iloc[test_idx].reset_index(drop=True)

y_dev    = dev_df["relevance"].values
y_test   = test_df["relevance"].values
grp_dev  = dev_df["search_term"]
grp_test = test_df["search_term"]

print(f"dev rows: {dev_df.shape}    test rows: {test_df.shape}")

dev rows: (590, 46)    test rows: (151, 46)


In [93]:
def infer_max_splits_from_df(df, group_col, test_size=0.2, min_test_groups=1):
    """
    Infer the max number of GroupShuffleSplit-compatible splits 
    based on the number of unique groups in a dataframe.

    Parameters:
        df (pd.DataFrame): Your full dataframe.
        group_col (str): Column name containing the grouping identifier.
        test_size (float): Fraction of groups to assign to test per split.
        min_test_groups (int): Minimum number of groups required in test set.

    Returns:
        int: Estimated safe max number of disjoint splits.
    """
    unique_groups = df[group_col].nunique()
    test_group_count = int(unique_groups * test_size)

    if test_group_count < min_test_groups:
        raise ValueError("Too few groups to form a meaningful test set.")

    max_disjoint_splits = unique_groups // test_group_count

    return max_disjoint_splits


In [94]:
print(f'Max disjoint splits possible: {infer_max_splits_from_df(dev_df, "search_term", test_size=0.2, min_test_groups=5)}')

Max disjoint splits possible: 5


#### Critical Pipeline encapsulation
1) Bayesian hyper parameter optimization
2) Cross validation to train the model and obtain RMSE distribution

In [95]:
# --------------------- helper utilities -------------------------
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def _cast_int(d):
    for k, v in list(d.items()):
        if isinstance(v, float) and v.is_integer():
            d[k] = int(v)
    return d

def _xgb_supports(arg: str) -> bool:
    return arg in inspect.signature(xgb.XGBRegressor.fit).parameters

# =================================================================
class MultiModelPipeline:
    # -------------------------------------------------------------
    def __init__(self, full_df: pd.DataFrame,
                 target_col="relevance",
                 group_col="search_term",
                 rs: int = 42):
        self.full   = full_df.copy().reset_index(drop=True)
        self.target = target_col
        self.groups = self.full[group_col]
        self.RS     = rs

        self.best_params = {}   # model → param dict
        self.split_rmses = {}   # model → list[rmse]

    # ------------------ search space ----------------------------
    @staticmethod
    def _space():
        return {
            "lgb": {
                "num_leaves": hp.quniform("lgb_num_leaves", 16, 512, 8),
                "max_depth": hp.quniform("lgb_max_depth", 4, 50, 1),
                "learning_rate": hp.loguniform("lgb_lr", np.log(0.005), np.log(0.15)),
                "n_estimators": hp.quniform("lgb_n", 400, 1600, 100),
                "min_child_samples": hp.quniform("lgb_mcs", 3, 80, 5),
                "boosting_type": hp.choice("lgb_boost", ["gbdt", "dart"]),
                "subsample": hp.uniform("lgb_sub", 0.5, 1.0),
                "colsample_bytree": hp.uniform("lgb_col", 0.5, 1.0),
                "subsample_freq": hp.quniform("lgb_freq", 1, 7, 1),
                "reg_alpha": hp.loguniform("lgb_alpha", np.log(1e-3), np.log(50)),
                "reg_lambda": hp.loguniform("lgb_lambda", np.log(1e-3), np.log(50)),
            },
            "xgb": {
                "max_depth": hp.quniform("xgb_depth", 3, 20, 1),
                "learning_rate": hp.loguniform("xgb_lr", np.log(0.005), np.log(0.3)),
                "n_estimators": hp.quniform("xgb_n", 400, 2000, 100),
                "subsample": hp.uniform("xgb_sub", 0.5, 1.0),
                "colsample_bytree": hp.uniform("xgb_col", 0.5, 1.0),
                "gamma": hp.uniform("xgb_gamma", 0, 5),
                "reg_alpha": hp.loguniform("xgb_alpha", np.log(1e-3), np.log(50)),
                "reg_lambda": hp.loguniform("xgb_lambda", np.log(1e-3), np.log(50)),
            },
            "cat": {
                "iterations": hp.quniform("cat_iter", 500, 3000, 200),
                "depth": hp.quniform("cat_depth", 4, 14, 1),
                "learning_rate": hp.loguniform("cat_lr", np.log(0.005), np.log(0.3)),
                "l2_leaf_reg": hp.loguniform("cat_l2", np.log(1e-3), np.log(50)),
                "subsample": hp.uniform("cat_sub", 0.5, 1.0),
                "bagging_temperature": hp.uniform("cat_bt", 0, 1),
            },
        }

    # ------------------ Inner CV Hyper Parameter tuning -------------------------
    def _tune(self, key, max_evals):
        space = self._space()[key]
        gkf   = GroupKFold(n_splits=5)

        def objective(p):
            p = _cast_int(p)
            rmses, rounds = [], []

            for tr_idx, va_idx in gkf.split(self.full, groups=self.groups):
                tr_df, va_df = self.full.iloc[tr_idx], self.full.iloc[va_idx]
                vec = {}
                X_tr = featurise(tr_df, fit=True,  vec_store=vec)
                X_va = featurise(va_df, fit=False, vec_store=vec)
                y_tr, y_va = tr_df[self.target], va_df[self.target]

                if key == "lgb":
                    mdl = lgb.LGBMRegressor(**p, objective="regression",
                                            metric="rmse", random_state=self.RS,
                                            n_jobs=-1)
                    mdl.fit(X_tr, y_tr, eval_set=[(X_va, y_va)],
                            eval_metric="rmse",
                            callbacks=[lgb.early_stopping(50)])
                    rounds.append(mdl.best_iteration_)
                elif key == "xgb":
                    mdl = xgb.XGBRegressor(**p, objective="reg:squarederror",
                                           random_state=self.RS,
                                           tree_method="hist", n_jobs=-1)
                    fit_kw = dict(eval_set=[(X_va, y_va)], verbose=False)
                    if _xgb_supports("callbacks"):
                        fit_kw["callbacks"] = [xgb.callback.EarlyStopping(50)]
                    elif _xgb_supports("early_stopping_rounds"):
                        fit_kw["early_stopping_rounds"] = 50
                    mdl.fit(X_tr, y_tr, **fit_kw)
                    rounds.append(getattr(mdl, "best_iteration", p["n_estimators"]))
                else:
                    mdl = CatBoostRegressor(**p, loss_function="RMSE",
                                            random_seed=self.RS, verbose=False)
                    mdl.fit(X_tr, y_tr, eval_set=(X_va, y_va),
                            early_stopping_rounds=50, use_best_model=True)
                    rounds.append(mdl.get_best_iteration())

                rmses.append(rmse(y_va, mdl.predict(X_va)))

            return {"loss": np.mean(rmses),
                    "status": STATUS_OK,
                    "best_round": int(np.mean(rounds))}

        trials = Trials()
        best   = fmin(objective, space, algo=tpe.suggest,
                      max_evals=max_evals, trials=trials,
                      rstate=np.random.default_rng(self.RS))
        bp = space_eval(space, best)
        br = trials.best_trial["result"]["best_round"]
        bp["n_estimators" if key != "cat" else "iterations"] = int(br)
        self.best_params[key] = bp

    # ------------------ Outer CV fit + score -----------------------
    def _model_rmse(self, key, params, X_tr, y_tr, X_va, y_va):
        params = _cast_int(params)
        """Train a single model on pre‑built feature matrices and return RMSE."""
        if key == "lgb":
            mdl = lgb.LGBMRegressor(**params, objective="regression",
                                    random_state=self.RS, n_jobs=-1)
        elif key == "xgb":
            mdl = xgb.XGBRegressor(**params, objective="reg:squarederror",
                                   random_state=self.RS, tree_method="hist",
                                   n_jobs=-1)
        else:
            mdl = CatBoostRegressor(**params, loss_function="RMSE",
                                    random_seed=self.RS, verbose=False)
        mdl.fit(X_tr, y_tr)
        return rmse(y_va, mdl.predict(X_va))
    # ------------------ public driver ---------------------------
    def run_all(self, keys=None, *, hpo_evals=40, repeats=10):
        keys = keys or list(self._space().keys())

        # 1) inner tuning
        for k in keys:
            self._tune(k, hpo_evals)
            self.split_rmses[k] = []

        # 2) Outer evaluation (sequential)
        gss = GroupShuffleSplit(n_splits=5 * repeats, test_size=0.2,
                                random_state=self.RS + 123)

        for tr_idx, va_idx in gss.split(self.full, groups=self.groups):
            # --- build features once for this split ------------------
            tr_df, va_df = self.full.iloc[tr_idx], self.full.iloc[va_idx]
            vec = {}
            X_tr = featurise(tr_df, fit=True,  vec_store=vec)
            X_va = featurise(va_df, fit=False, vec_store=vec)
            y_tr, y_va = tr_df[self.target].values, va_df[self.target].values

            # --- evaluate each model on the same matrices ------------
            for k in keys:
                rmse_val = self._model_rmse(k,
                                            self.best_params[k],
                                            X_tr, y_tr, X_va, y_va)
                self.split_rmses[k].append(rmse_val)
     # ------------------ bootstrap error dist --------------------
    def bootstrap_distribution(self, B=1000, random_state=42):
        """
        Returns a dictionary of bootstrapped mean RMSEs (length=B)
        for each model using the outer CV split_rmses.
        """
        rng = np.random.default_rng(random_state)
        boot = {}

        for model, errors in self.split_rmses.items():
            errors = np.asarray(errors)
            n = len(errors)
            idx = rng.integers(0, n, size=(B, n))
            boot[model] = errors[idx].mean(axis=1)

        return boot
    # ------------------ export ----------------------------------
    def export_files(self, out_dir="results", bs_size=200):
        os.makedirs(out_dir, exist_ok=True)
        with open(os.path.join(out_dir, "best_params.json"), "w") as fp:
            json.dump(self.best_params, fp, indent=2)

        rows = ({"model": m, "split_no": i, "rmse": v}
                for m, lst in self.split_rmses.items()
                for i, v in enumerate(lst, 1))
        pd.DataFrame(rows).to_csv(
            os.path.join(out_dir, "rmse_distribution.csv"), index=False
        )
        # Export bootstrapped distribution
        boot = self.bootstrap_distribution(B = bs_size, random_state=123)
        rows = ({"model": m, "resample_no": i, "rmse": v}
                for m, vals in boot.items()
                for i, v in enumerate(vals, 1))
        pd.DataFrame(rows).to_csv(
            os.path.join(out_dir, "rmse_bootstrap.csv"), index=False
        )
        return out_dir
    

#### Pipeline Execution

In [96]:
pipe = MultiModelPipeline(dev_df)
pipe.run_all(keys=["xgb","cat","lgb"], hpo_evals = 1, repeats=1)
# I have provided the real parameters and rmse values based on the whole dataset. Still, if you want to reporduce the results,
# please uncomment the line below and use all the dataset rather than just a sample (computationally expensive) 
#pipe.export_files("./object/", bs_size=200)

100%|██████████| 1/1 [02:19<00:00, 139.24s/trial, best loss: 0.5143042904460444]
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000477 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1319                    
[LightGBM] [Info] Number of data points in the train set: 472, number of used features: 30
[LightGBM] [Info] Start training from score 2.369322 
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

#### Select Model

In [97]:
s_opath = "./object/"
df_rmses = pd.read_csv(s_opath+"rmse_distribution.csv")
df_rmses_bs = pd.read_csv(s_opath+"rmse_bootstrap.csv")

In [98]:
# Summary statistics
summary_df = df_rmses.groupby("model")["rmse"].agg(["mean", "std"]).reset_index()
summary_df.columns = ["Model", "Mean RMSE", "Std RMSE"]
# Summary statistics
summary_df_bs = df_rmses_bs.groupby("model")["rmse"].agg(["mean", "std"]).reset_index()
summary_df_bs.columns = ["Model", "Mean RMSE", "Std RMSE"]
display(summary_df, summary_df_bs)

,Model,Mean RMSE,Std RMSE
0,cat,0.489957,0.021322
1,lgb,0.489112,0.023977
2,xgb,0.490582,0.022820


,Model,Mean RMSE,Std RMSE
0,cat,0.490687,0.008637
1,lgb,0.488817,0.010264
2,xgb,0.490146,0.009714


In [99]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplot layout
fig = make_subplots(rows=1, cols=2, subplot_titles=("CV Splits RMSE", "Bootstrapped RMSE"))

# Define model colors
color_map = {'lgb': 'green', 'xgb': 'blue', 'cat': 'red'}

# First plot: CV splits
for model in df_rmses["model"].unique():
    fig.add_trace(
        go.Box(
            y=df_rmses[df_rmses["model"] == model]["rmse"],
            name=model,
            marker_color=color_map.get(model, "gray"),
            boxmean="sd"
        ),
        row=1, col=1
    )

# Second plot: Bootstrapped
for model in df_rmses_bs["model"].unique():
    fig.add_trace(
        go.Box(
            y=df_rmses_bs[df_rmses_bs["model"] == model]["rmse"],
            name=model,
            marker_color=color_map.get(model, "gray"),
            boxmean="sd",
            showlegend=False
        ),
        row=1, col=2
    )

# Layout adjustments
fig.update_layout(
    height=400,
    width=850,
    title_text="RMSE Distributions by Model (CV Splits vs Bootstrapped)"
)

fig.update_yaxes(title_text="RMSE", row=1, col=1)
fig.update_yaxes(title_text="RMSE", row=1, col=2)
fig.update_xaxes(title_text="Model", row=1, col=1)
fig.update_xaxes(title_text="Model", row=1, col=2)

fig.show()


#### Evaluate Performance

In [100]:
with open(s_opath+"best_params.json", "r") as f:
	d_p = json.load(f)
d_params = d_p['cat']

In [101]:
d_params = _cast_int(d_params)
X_tr = featurise(dev_df, fit=True)
X_va, feature_names = featurise(test_df, fit=False, return_columns=True) #featurise and get feature names

In [102]:
#with open(os.devnull, 'w') as f, redirect_stdout(f):
key_mdl = CatBoostRegressor(**d_params, loss_function="RMSE",
                                    random_seed = RS_STATE, verbose=False)
key_mdl.fit(X_tr, y_dev)
mdl_rmse_test = rmse(y_test, key_mdl.predict(X_va))
mdl_rmse_train = rmse(y_dev, key_mdl.predict(X_tr))
print('Completed!')    

Completed!


In [103]:
print(f'The RMSE of the best model - CatGBoost - is: {mdl_rmse_test:.4f}')

The RMSE of the best model - CatGBoost - is: 0.5212


In [104]:
df_results = pd.DataFrame([{
    "Model": "CatGBoost",
    "Score (Train)": mdl_rmse_train,
    "Score (Test)": mdl_rmse_test
}])

df_results

,Model,Score (Train),Score (Test)
0,CatGBoost,0.470606,0.521209


In [105]:
def single_model_bootstrap_test(df_obs: pd.DataFrame, df_boot: pd.DataFrame,
                                model: str, alpha: float = 0.05):
    """Return a dict with observed mean, 95 % CI and p‑value for one model."""
    obs_vals   = df_obs.loc[df_obs["model"] == model, "rmse"].values
    boot_means = df_boot.loc[df_boot["model"] == model, "rmse"].values

    obs_mean   = obs_vals.mean()
    boot_mean  = boot_means.mean()
    # two‑tailed percentile CI for the mean
    ci_low, ci_high = np.quantile(boot_means, [alpha/2, 1 - alpha/2])

    # p‑value: how often is |boot_mean_i − boot_mean| ≥ |obs_mean − boot_mean| ?
    thresh = abs(obs_mean - boot_mean)
    p_val  = (np.abs(boot_means - boot_mean) >= thresh).mean()

    return {
        "Model": model,
        "Observed mean": obs_mean,
        "Boot mean": boot_mean,
        "95% CI low": ci_low,
        "95% CI high": ci_high,
        "p‑value": p_val,
    }


In [106]:
#  For each model m we test
#      H0 : The expected mean RMSE = the mean observed on the 5 splits  (no bias)
#      HA : The expected mean RMSE ≠ that value
results = [single_model_bootstrap_test(df_rmses, df_rmses_bs, m )
           for m in df_rmses["model"].unique()]

summary = pd.DataFrame(results).sort_values("Model")
print("Per‑model bootstrap consistency test (two‑tailed):")
display(summary)

Per‑model bootstrap consistency test (two‑tailed):


,Model,Observed mean,Boot mean,95% CI low,95% CI high,p‑value
1,cat,0.489957,0.490687,0.474291,0.505794,0.960
2,lgb,0.489112,0.488817,0.471505,0.506772,0.980
0,xgb,0.490582,0.490146,0.470735,0.506415,0.965


In [107]:
import plotly.express as px

def plot_catboost_importance(model, feature_names, top_n: int = 25):
    """Horizontal bar chart of CatBoost feature importances."""
    importances = model.get_feature_importance(type="FeatureImportance")
    df_imp = pd.DataFrame({"feature": feature_names, "importance": importances})
    df_imp = df_imp.sort_values("importance", ascending=False).head(top_n)

    fig = px.bar(df_imp[::-1], x="importance", y="feature", orientation="h",
                 title=f"CatBoost feature importance (top {top_n})")
    fig.update_layout(yaxis={'categoryorder': 'total ascending'},
                      xaxis_title="Importance score",
                      yaxis_title="Feature")
    fig.show()
    return df_imp


In [108]:
plot_catboost_importance(key_mdl, feature_names, top_n=20)

,feature,importance
0,cos_t,25.012903
3,bm25_l,23.895064
4,cos_char,9.236047
1,cos_l,9.078085
2,bm25_t,6.242191
10,d_min,3.626904
13,d_span,2.729530
11,d_max,2.695987
5,t_min,1.720505
8,t_span,1.695189


#### CLUSTERING

Disclaimer: Clustering was performed exclusively with the training / dev_df dataset. My overall approach consists in disregarding the testing set except for model performance evaluations to avoid bias in all the inferences.

Since we are using a sample dataset to test that the workflow work properly, the clustering results will differ from the real values presented in the report.
Still, you can change the Sample DF section and obtain the real results (computationally expensive) 

In [ ]:
from sklearn.decomposition import TruncatedSVD
def zero_impute(X):
    """
    Replace every NaN with 0 in a pandas.DataFrame, a NumPy ndarray,
    or any SciPy sparse matrix (csr, csc, coo, …).
    Returns the cleaned object in the same format it received.
    """
    # -------- 1) Pandas DataFrame ----------------------------------------
    if isinstance(X, pd.DataFrame):
        return X.fillna(0)

    # -------- 2) SciPy sparse matrix -------------------------------------
    if sp.issparse(X):
        # Work on the data vector only → keeps the matrix sparse
        nan_mask = np.isnan(X.data)
        if nan_mask.any():
            X.data[nan_mask] = 0.0
        return X   # still the same sparse format (csr, csc, …)

    # -------- 3) Plain NumPy array ---------------------------------------
    return np.nan_to_num(X, nan=0.0)

"""
workflow summary:
------------------------------
1) t-SNE  (tunes perplexity + learning_rate)
2) Hyperopt  -> best (perplexity, lr, k) w.r.t. Silhouette
3) KMeans fit on full data with best k
4) Word-cloud of top 10 search-term tokens / cluster
"""

# ---------- INPUTS ------------------------------------------------------
# X_train  : your dense feature matrix (n × d)
# train    : DataFrame with "search_term" column
# ------------------------------------------------------------------------

# Zero impute
X_train = zero_impute(X_tr)
assert not np.isnan(
    X_train.data if sp.issparse(X_train) else X_train
).any(), "There are still NaNs!"

# Dimensionality reduction
svd = TruncatedSVD(n_components=50, random_state=RS_STATE)
X_reduced = svd.fit_transform(X_train)   # dense (n_samples × 50)
X_train = X_reduced

# Subsample for speed
MAX_PTS_FOR_TSNE = 10_000            
idx = np.arange(X_train.shape[0])
if MAX_PTS_FOR_TSNE and len(idx) > MAX_PTS_FOR_TSNE:
    idx = np.random.choice(idx, MAX_PTS_FOR_TSNE, replace=False)
X_sub = X_train[idx]

#Hyperopt for best t-SNE + Kmeans params
space = {
    "perplexity":     hp.quniform("perplexity", 10, 60, 5),
    "learning_rate":  hp.loguniform("learning_rate", np.log(50), np.log(500)),
    "k":              hp.quniform("k", 3, 15, 1),
}

def objective(params):
    perpl = float(params["perplexity"])
    lr    = float(params["learning_rate"])
    k     = int(params["k"])

    tsne = TSNE(n_components=2, perplexity=perpl, learning_rate=lr,
                n_iter=250, init="random", random_state=RS_STATE)
    embedded = tsne.fit_transform(X_sub)

    km = KMeans(n_clusters=k, random_state=RS_STATE, n_init="auto")
    labels = km.fit_predict(embedded)

    sil = silhouette_score(embedded, labels)
    return {"loss": -sil, "status": STATUS_OK, "sil": sil}

trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest,
            max_evals=50, trials=trials, rstate=np.random.default_rng(42))

best_perp = float(best["perplexity"])
best_lr   = float(best["learning_rate"])
best_k    = int(best["k"])
best_sil  = -min(t["result"]["loss"] for t in trials.trials)

print(f"\n  Best params  →  perplexity={best_perp}, lr={best_lr:.1f}, k={best_k}")
print(f"  Best Silhouette on subsample: {best_sil:.4f}")

# ---------- Final Embed in ALL data with best t-SNE ------------------------------
tsne_final = TSNE(n_components=2, perplexity=best_perp, learning_rate=best_lr,
                  n_iter=250, init="random", random_state=RS_STATE)
all_emb = tsne_final.fit_transform(X_train)

# ---------- Final KMeans -------------------------------------------------
km_final   = KMeans(n_clusters=best_k, random_state=RS_STATE, n_init="auto")
labels_all = km_final.fit_predict(all_emb)

sil_full = silhouette_score(all_emb, labels_all)
print(f"Silhouette (all data): {sil_full:.4f}")

# ---------- Top-10 search-term tokens per cluster -----------------------
vectorizer   = CountVectorizer(stop_words="english")
term_matrix  = vectorizer.fit_transform(dev_df["search_term"])
vocab        = np.array(vectorizer.get_feature_names_out())
top_n        = 10
cluster_words= {}

for c in range(best_k):
    rows = np.where(labels_all == c)[0]
    if rows.size == 0:
        continue
    counts   = np.asarray(term_matrix[rows].sum(axis=0)).ravel()
    top_idx  = counts.argsort()[::-1][:top_n]
    cluster_words[c] = list(vocab[top_idx])
    print(f"Cluster {c}: {', '.join(cluster_words[c])}")



100%|██████████| 50/50 [02:30<00:00,  3.00s/trial, best loss: -0.9660619497299194]

  Best params  →  perplexity=40.0, lr=76.4, k=3
  Best Silhouette on subsample: 0.9661
Silhouette (all data): 0.9661
Cluster 0: light, door, shower, tile, inch, wall, wood, white, glass, fence
Cluster 1: planks, pine, 10, zero, zep, zenith, year, x4, x34, wrenches
Cluster 2: round, outdoor, cushions, zero, zep, zenith, year, x4, x34, wrenches


In [111]:
# ------------------------ 7. Export top-10 cluster words to CSV ------------------------
# Convert cluster_words dict to a DataFrame
cluster_words_df = pd.DataFrame.from_dict(cluster_words, orient="index")
cluster_words_df.columns = [f"top_word_{i+1}" for i in range(cluster_words_df.shape[1])]
cluster_words_df.index.name = "cluster_id"

# Export to CSV
#cluster_words_df.to_csv("./object/top_words_per_cluster.csv", encoding="utf-8", index=True)

In [112]:
import plotly.express as px
import plotly.graph_objects as go
from PIL import Image
import io
import plotly.io as pio



# ------------------------------------------------------------------
# 1) Interactive scatterplot of the t-SNE embedding
# ------------------------------------------------------------------
fig_scatter = px.scatter(
    x=all_emb[:, 0],
    y=all_emb[:, 1],
    color=labels_all.astype(str),        # colour by cluster id
    title="t-SNE embedding (best params) – coloured by K-Means cluster",
    labels={"x": "t-SNE-1", "y": "t-SNE-2", "color": "cluster"},
    width=900,
    height=700
)
fig_scatter.update_traces(marker=dict(size=5, line=dict(width=0)))
fig_scatter.show()

# ------------------------------------------------------------------
# 2) Word-cloud per cluster rendered in Plotly
# ------------------------------------------------------------------
for c, words in cluster_words.items():
    wc = WordCloud(
        background_color="white",
        width=700,
        height=350,
        colormap="tab10"
    ).generate(" ".join(words))

    # Convert PIL image to bytes buffer
    buf = io.BytesIO()
    wc.to_image().save(buf, format="PNG")
    buf.seek(0)

    # Build a figure with a single image trace
    fig_wc = go.Figure()
    fig_wc.add_layout_image(
        dict(
            source=Image.open(buf),
            xref="x", yref="y",
            x=0, y=1,        # position
            sizex=1, sizey=1,
            sizing="stretch",
            layer="below"
        )
    )
    fig_wc.update_xaxes(visible=False)
    fig_wc.update_yaxes(visible=False)
    fig_wc.update_layout(
        width=700, height=350,
        title=f"Cluster {c} – top tokens",
        margin=dict(l=0, r=0, t=40, b=0)
    )
    fig_wc.show()
